In [1]:
import pandas as pd
import numpy as np

In [2]:
df_train = pd.read_parquet("../Dataset/NF-ToN-IoT/NF-ToN-IoT-V2_train.parquet")

In [3]:
df_valid = pd.read_parquet("../Dataset/NF-ToN-IoT/NF-ToN-IoT-V2_valid.parquet")

In [4]:
df_test = pd.read_parquet("../Dataset/NF-ToN-IoT/NF-ToN-IoT-V2_test.parquet")

In [5]:
load_X_df_train = df_train.drop(["Label"],axis=1)
Y_df_train = df_train["Label"]

In [6]:
load_X_df_valid = df_valid.drop(["Label"],axis=1)
Y_df_valid = df_valid["Label"]

In [7]:
load_X_df_test = df_test.drop(["Label"],axis=1)
Y_df_test = df_test["Label"]

In [8]:
from sklearn.preprocessing import QuantileTransformer

In [9]:
scaler_df = QuantileTransformer(output_distribution='normal')

In [10]:
X_df_train = scaler_df.fit_transform(load_X_df_train)

In [11]:
X_df_valid = scaler_df.fit_transform(load_X_df_valid)

In [12]:
X_df_test = scaler_df.fit_transform(load_X_df_test)

In [13]:
from keras.models import Sequential
from keras.layers import Dense

2025-11-17 17:19:52.460761: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-17 17:19:52.464111: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-17 17:19:52.471978: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-11-17 17:19:52.486831: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-11-17 17:19:52.490867: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-11-17 17:19:52.502732: I tensorflow/core/platform/cpu_feature_gu

In [14]:
from keras.metrics import Recall, Precision

In [15]:
def fit_model(trainX, trainy):
    model = Sequential(name="N2_model")
    model.add(Dense(9, input_dim=39, activation='relu', name="N2_i"))
    model.add(Dense(7, activation='relu', name="N2_l1"))
    model.add(Dense(5, activation='relu', name="N2_l2"))
    model.add(Dense(3, activation='relu', name="N2_l3"))
    model.add(Dense(1, activation='sigmoid', name="N2_o")) 
    model._name="N2_m"
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', Recall(), Precision()])
    
    model.fit(trainX, trainy, epochs=10, batch_size=1000, verbose=1, validation_data=(X_df_valid, Y_df_valid))
    return model

In [16]:
model = fit_model(X_df_train, Y_df_train)

filename = './model_N2.keras'
model.save(filename)
print('>Saved %s' % filename)

/home/awardana/miniconda3/envs/cids_env/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
8276/8276 ━━━━━━━━━━━━━━━━━━━━ 11s 1ms/step - accuracy: 0.8990 - loss: 0.2215 - precision: 0.9030 - recall: 0.9704 - val_accuracy: 0.9693 - val_loss: 0.0800 - val_precision: 0.9754 - val_recall: 0.9824
Epoch 2/10
8276/8276 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms/step - accuracy: 0.9695 - loss: 0.0789 - precision: 0.9746 - recall: 0.9838 - val_accuracy: 0.9717 - val_loss: 0.0763 - val_precision: 0.9787 - val_recall: 0.9824
Epoch 3/10
8276/8276 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms/step - accuracy: 0.9714 - loss: 0.0751 - precision: 0.9763 - recall: 0.9846 - val_accuracy: 0.9730 - val_loss: 0.0729 - val_precision: 0.9756 - val_recall: 0.9874
Epoch 4/10
8276/8276 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms/step - accuracy: 0.9723 - loss: 0.0731 - precision: 0.9769 - recall: 0.9851 - val_accuracy: 0.9714 - val_loss: 0.0733 - val_precision: 0.9745 - val_recall: 0.9864
Epoch 5/10
8276/8276 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms/step - accuracy: 0.9725 - loss: 0.0720 - precision: 0.9771 - recall: 0.9852 - val_accuracy: 0.9732

In [17]:
from keras.models import load_model
from numpy import argmax
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

In [18]:
filename_model = './model_N2.keras'
loaded_model = load_model(filename_model)

In [19]:
score = loaded_model.evaluate(X_df_test, Y_df_test, verbose=1)

print('Test loss:', score[0])
print('Test accuracy:', score[1])
print('Test recall:', score[2])
print('Test precision:', score[3])

# Calculate F1-score
precision = score[3]
recall = score[2]
f1_score = 2 * (precision * recall) / (precision + recall)

print('Test F1-score:', f1_score)


110834/110834 ━━━━━━━━━━━━━━━━━━━━ 85s 767us/step - accuracy: 0.8618 - loss: 0.8198 - precision: 0.9730 - recall: 0.8329
Test loss: 0.8185136914253235
Test accuracy: 0.8619495034217834
Test recall: 0.8329985737800598
Test precision: 0.9729402661323547
Test F1-score: 0.8975473987820644
